In [1]:
# Импортируем необходимые библиотеки 

import pandas as pd
import os
from PIL import Image
from pycocotools import mask as maskUtils
import torch
import torchvision
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
import cv2
import numpy as np
from sklearn import tree
from sklearn import ensemble


In [18]:
# Собираем данные об объектах и и сохраняем получившийся DataFrame в формате CSV
# Загрузка предобученной модели Mask R-CNN
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Путь к папке с изображениями
image_folder = "/home/tatiana/Документы/SkillFactoryTask/Интересное/archive/furniture_images/furniture_images"
image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]

# Список аннотаций
annotations = []

# Идентификаторы
image_id = 1
annotation_id = 1

# Перебор изображений
for image_file in image_files:
    # Проверка на количество объектов
    if annotation_id > 9346:
        break
    
    # Загрузка и предобработка изображения
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)
    transform = transforms.Compose([transforms.ToTensor()])
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)

    # Передача изображения через модель
    with torch.no_grad():
        outputs = model(image_tensor)

    # Получение характеристик объектов
    boxes = outputs[0]['boxes'].detach().numpy()  # Ограничивающие прямоугольники
    labels = outputs[0]['labels'].detach().numpy()  # Категории объектов
    scores = outputs[0]['scores'].detach().numpy()  # Оценки уверенности
    masks = outputs[0]['masks'].detach().numpy()  # Маски сегментации
    
    # Получение ширины и высоты изображения
    width, height = image.size

    # Вывод характеристик объектов
    for box, label, score, mask in zip(boxes, labels, scores, masks):
        if annotation_id > 9346:
            break
        
        # Создание аннотации в формате COCO
        annotation = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": label.item(),
            "bbox": [box[0].item(), box[1].item(), (box[2] - box[0]).item(), (box[3] - box[1]).item()],
            "segmentation": [],  # Изменить значение сегментации
            "area": ((box[2] - box[0]).item()) * ((box[3] - box[1]).item()),
            "file_name": image_file,
            "width": width,
            "height": height,
            "attributes": {  # Добавить словарь атрибутов
                "color": "red",  # Пример атрибута цвета
                "size": "small", # Пример атрибута размера
                "material": "wood" 

            }
        }
        
        # Получение координат точек или контуров из маски сегментации
        if mask.ndim == 3:  # Если маска имеет три канала (RGB)
            mask = np.sum(mask, axis=0)  # Объединить каналы маски
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                contour = contour.flatten().tolist()
                annotation["segmentation"].append(contour)
        
        # Добавление аннотации в список
        annotations.append(annotation)

        # Увеличение идентификаторов
        annotation_id += 1

    image_id += 1

# Создание DataFrame из списка аннотаций
df_annotations = pd.DataFrame(annotations)

# Сохранение DataFrame в формате CSV
df_annotations.to_csv("annotations.csv", index=False)

/home/tatiana/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tatiana/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# Читаем файл
df_annotations = pd.read_csv("annotations.csv")
df_annotations.head()


,id,image_id,category_id,bbox,segmentation,area,file_name,width,height,attributes
0,1,1,84,"[96.75932312011719, 19.740680694580078, 7.2847...",[],156.962108,1634018288121_Small Cupboard Melamine for sale...,142,107,"{'color': 'red', 'size': 'small', 'material': ..."
1,2,1,84,"[99.91023254394531, 19.759830474853516, 11.340...",[],257.873625,1634018288121_Small Cupboard Melamine for sale...,142,107,"{'color': 'red', 'size': 'small', 'material': ..."
2,3,1,15,"[62.290489196777344, 48.610076904296875, 48.81...",[],1357.352472,1634018288121_Small Cupboard Melamine for sale...,142,107,"{'color': 'red', 'size': 'small', 'material': ..."
3,4,1,44,"[97.18596649169922, 20.677946090698242, 4.9057...",[],99.229718,1634018288121_Small Cupboard Melamine for sale...,142,107,"{'color': 'red', 'size': 'small', 'material': ..."
4,5,2,72,"[0.15689216554164886, 47.815216064453125, 9.06...",[],387.857851,1634028472376_3 Door Cupboard for sale.jpg,142,107,"{'color': 'red', 'size': 'small', 'material': ..."


In [3]:
# Создаем дополнительный признак token из названия файла. из него мы получим названия предметов мебели для разметки данных
import re
df_annotations['tokens'] = df_annotations['file_name'].str.lower().str.split()
df_annotations

,id,image_id,category_id,bbox,segmentation,area,file_name,width,height,attributes,tokens
0,1,1,84,"[96.75932312011719, 19.740680694580078, 7.2847...",[],156.962108,1634018288121_Small Cupboard Melamine for sale...,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634018288121_small, cupboard, melamine, for,..."
1,2,1,84,"[99.91023254394531, 19.759830474853516, 11.340...",[],257.873625,1634018288121_Small Cupboard Melamine for sale...,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634018288121_small, cupboard, melamine, for,..."
2,3,1,15,"[62.290489196777344, 48.610076904296875, 48.81...",[],1357.352472,1634018288121_Small Cupboard Melamine for sale...,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634018288121_small, cupboard, melamine, for,..."
3,4,1,44,"[97.18596649169922, 20.677946090698242, 4.9057...",[],99.229718,1634018288121_Small Cupboard Melamine for sale...,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634018288121_small, cupboard, melamine, for,..."
4,5,2,72,"[0.15689216554164886, 47.815216064453125, 9.06...",[],387.857851,1634028472376_3 Door Cupboard for sale.jpg,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634028472376_3, door, cupboard, for, sale.jpg]"
...,...,...,...,...,...,...,...,...,...,...,...
9341,9342,387,1,"[133.1674346923828, 39.89961242675781, 8.09828...",[],241.948095,1634019989697_Teak Bedroom SetPackage Kingsize...,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634019989697_teak, bedroom, setpackage, king..."
9342,9343,387,62,"[82.98673248291016, 75.73381805419922, 33.3057...",[],990.762539,1634019989697_Teak Bedroom SetPackage Kingsize...,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634019989697_teak, bedroom, setpackage, king..."
9343,9344,387,81,"[14.994020462036133, 82.60379791259766, 9.1288...",[],85.797444,1634019989697_Teak Bedroom SetPackage Kingsize...,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634019989697_teak, bedroom, setpackage, king..."
9344,9345,387,3,"[47.196800231933594, 77.2764663696289, 17.6679...",[],241.356833,1634019989697_Teak Bedroom SetPackage Kingsize...,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634019989697_teak, bedroom, setpackage, king..."


In [4]:
# Список предметов мебели
furn = ['cupboard', 'bed', 'table', 'sofa', 'box',  'chair', 'towel', 'frame', 'mattress', 'wardrobe', 'stand']

In [48]:

# Создание целевого признака 'goal'
df_annotations['goal'] = df_annotations['tokens'].apply(lambda x: next((word for word in x if word in furn), None))

# Вывод DataFrame с значением goal = None
filtered_df = df_annotations.loc[df_annotations['goal'].isnull()]

# Вывод отфильтрованного DataFrame
filtered_df.head()

,id,image_id,category_id,bbox,segmentation,area,file_name,width,height,attributes,tokens,goal
6,7,3,1,"[103.92316436767578, 15.26392650604248, 38.076...",[],3266.819764,1634022055458_Plastic chairs for sale.jpg,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634022055458_plastic, chairs, for, sale.jpg]",None
7,8,3,1,"[5.938572406768799, 0.0, 103.86293029785156, 1...",[],11048.536487,1634022055458_Plastic chairs for sale.jpg,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634022055458_plastic, chairs, for, sale.jpg]",None
8,9,3,1,"[1.9201451539993286, 20.568634033203125, 60.51...",[],5147.852835,1634022055458_Plastic chairs for sale.jpg,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634022055458_plastic, chairs, for, sale.jpg]",None
9,10,3,1,"[39.505821228027344, 5.300530433654785, 98.551...",[],9676.752246,1634022055458_Plastic chairs for sale.jpg,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634022055458_plastic, chairs, for, sale.jpg]",None
10,11,3,17,"[33.41966247558594, 27.738998413085938, 82.823...",[],5188.014360,1634022055458_Plastic chairs for sale.jpg,142,107,"{'color': 'red', 'size': 'small', 'material': ...","[1634022055458_plastic, chairs, for, sale.jpg]",None


In [6]:
# удалим пустые значения
df = df_annotations.dropna()

In [7]:
df = df.reset_index(drop=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6450 entries, 0 to 6449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            6450 non-null   int64  
 1   image_id      6450 non-null   int64  
 2   category_id   6450 non-null   int64  
 3   bbox          6450 non-null   object 
 4   segmentation  6450 non-null   object 
 5   area          6450 non-null   float64
 6   file_name     6450 non-null   object 
 7   width         6450 non-null   int64  
 8   height        6450 non-null   int64  
 9   attributes    6450 non-null   object 
 10  tokens        6450 non-null   object 
 11  goal          6450 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 604.8+ KB


In [9]:
# Удалим пустые признаки 
df = df.drop(['segmentation', 'file_name', 'tokens', 'width', 'height', 'id', 'attributes'], axis=1)

In [10]:
# Создадим из списка bbox четыре новыз признака. Т.к в том виде как представлен признак bbox он не подходит для модели

df[['bbox1', 'bbox2', 'bbox3', 'bbox4']] = pd.DataFrame(df['bbox'].str.strip('[]').str.split(',').tolist()).astype(float)


In [11]:
# Удалим bbox
df = df.drop('bbox', axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6450 entries, 0 to 6449
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   image_id     6450 non-null   int64  
 1   category_id  6450 non-null   int64  
 2   area         6450 non-null   float64
 3   goal         6450 non-null   object 
 4   bbox1        6450 non-null   float64
 5   bbox2        6450 non-null   float64
 6   bbox3        6450 non-null   float64
 7   bbox4        6450 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 403.2+ KB


In [13]:
# Корреляция признаков
df.corr()

/tmp/ipykernel_308866/2828039058.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,image_id,category_id,area,bbox1,bbox2,bbox3,bbox4
image_id,1.000000,0.025842,-0.029705,-0.002401,-0.009582,-0.049057,-0.033395
category_id,0.025842,1.000000,-0.037586,-0.004906,0.033044,-0.019351,-0.040766
area,-0.029705,-0.037586,1.000000,-0.436687,-0.346707,0.885469,0.800476
bbox1,-0.002401,-0.004906,-0.436687,1.000000,0.103736,-0.488711,-0.297785
bbox2,-0.009582,0.033044,-0.346707,0.103736,1.000000,-0.200146,-0.412778
bbox3,-0.049057,-0.019351,0.885469,-0.488711,-0.200146,1.000000,0.586806
bbox4,-0.033395,-0.040766,0.800476,-0.297785,-0.412778,0.586806,1.000000


In [14]:
df['goal'].value_counts()

table       1760
sofa        1702
bed          764
box          708
chair        594
cupboard     458
mattress     249
stand        129
wardrobe      66
frame         15
towel          5
Name: goal, dtype: int64

In [28]:
# Закодируем целевой признак
# Создание экземпляра LabelEncoder
label_encoder = LabelEncoder()

# Применение Label Encoding к целевому признаку
df['goal_encoded'] = label_encoder.fit_transform(df['goal'])


In [29]:
df

,image_id,category_id,area,goal,bbox1,bbox2,bbox3,bbox4,goal_encoded
0,1,84,156.962108,cupboard,96.759323,19.740681,7.284760,21.546642,3
1,1,84,257.873625,cupboard,99.910233,19.759830,11.340218,22.739742,3
2,1,15,1357.352472,cupboard,62.290489,48.610077,48.818268,27.804192,3
3,1,44,99.229718,cupboard,97.185966,20.677946,4.905762,20.227179,3
4,2,72,387.857851,cupboard,0.156892,47.815216,9.066530,42.779083,3
...,...,...,...,...,...,...,...,...,...
6445,385,90,703.169262,box,18.649944,54.343536,17.000771,41.361023,1
6446,385,28,1164.770092,box,1.905130,53.690247,26.964363,43.196648,1
6447,385,6,134.924943,box,132.877441,92.112274,9.102097,14.823502,1
6448,385,58,633.078660,box,4.514712,61.093365,19.424313,32.592075,1


In [30]:
# Получение соответствия меток и исходных значений целевого признака
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Вывод соответствия меток и исходных значений
for label, value in label_mapping.items():
    print(f"Метка: {value}, Значение: {label}")

Метка: 0, Значение: bed
Метка: 1, Значение: box
Метка: 2, Значение: chair
Метка: 3, Значение: cupboard
Метка: 4, Значение: frame
Метка: 5, Значение: mattress
Метка: 6, Значение: sofa
Метка: 7, Значение: stand
Метка: 8, Значение: table
Метка: 9, Значение: towel
Метка: 10, Значение: wardrobe


In [31]:
# Создадим копию ДФ для дальнейшей работы
df_copy = df.copy()

In [32]:
# Удалим исходный незакодированный целевой
df_copy = df_copy.drop('goal', axis=1)

In [33]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6450 entries, 0 to 6449
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   image_id      6450 non-null   int64  
 1   category_id   6450 non-null   int64  
 2   area          6450 non-null   float64
 3   bbox1         6450 non-null   float64
 4   bbox2         6450 non-null   float64
 5   bbox3         6450 non-null   float64
 6   bbox4         6450 non-null   float64
 7   goal_encoded  6450 non-null   int64  
dtypes: float64(5), int64(3)
memory usage: 403.2 KB


In [34]:
# Разделим ДФ на Х - данные для обучения, у - целевой признак, который мы предсказываем
X = df_copy.drop("goal_encoded", axis=1)
y = df_copy["goal_encoded"]



In [35]:
# Разделим выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, 
    stratify=y, 
    random_state=0
)
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_test.shape))


Train shape: (4837, 7)
Test shape: (1613, 7)


## Логистическая регрессия

In [37]:
#Создаём объект класса LogisticRegression
log_reg = linear_model.LogisticRegression()
#Обучаем модель на тренировочной выборке
log_reg.fit(X_train, y_train)



/home/tatiana/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [38]:

# Обучим наши данные на модели с параметрами по умолчанию, для фиксирования начальных метрик
y_train_pred_log = log_reg.predict(X_train)
y_test_pred_log = log_reg.predict(X_test)


In [39]:
# Метрики
print(metrics.classification_report(y_train, y_train_pred_log))
print(metrics.classification_report(y_test, y_test_pred_log))

              precision    recall  f1-score   support

           0       0.22      0.11      0.14       573
           1       0.00      0.00      0.00       531
           2       0.33      0.07      0.12       445
           3       0.00      0.00      0.00       343
           4       0.00      0.00      0.00        11
           5       0.18      0.04      0.07       187
           6       0.30      0.42      0.35      1276
           7       0.00      0.00      0.00        97
           8       0.32      0.64      0.43      1320
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00        50

    accuracy                           0.31      4837
   macro avg       0.12      0.12      0.10      4837
weighted avg       0.23      0.31      0.24      4837

              precision    recall  f1-score   support

           0       0.20      0.10      0.13       191
           1       0.00      0.00      0.00       177
           2       0.23 

/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.1

Вывод: поскольку мы собрали данные только по расположению предмета в пространстве модель предсказуемо хорошо определяет только большие предметы: стол, софа, кровать.

In [40]:
# Нормализуем данные 
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [41]:
log_reg.fit(X_train_scaled, y_train)

/home/tatiana/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [42]:
y_train_pred_log_scaled = log_reg.predict(X_train_scaled)
y_test_pred_log_scaled = log_reg.predict(X_test_scaled)

In [47]:
print(metrics.classification_report(y_train, y_train_pred_log_scaled))
print(metrics.classification_report(y_test, y_test_pred_log_scaled))

              precision    recall  f1-score   support

           0       0.32      0.19      0.24       573
           1       0.33      0.00      0.00       531
           2       0.30      0.13      0.18       445
           3       0.00      0.00      0.00       343
           4       0.00      0.00      0.00        11
           5       0.50      0.01      0.02       187
           6       0.35      0.56      0.43      1276
           7       0.00      0.00      0.00        97
           8       0.36      0.61      0.45      1320
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00        50

    accuracy                           0.35      4837
   macro avg       0.20      0.14      0.12      4837
weighted avg       0.31      0.35      0.28      4837

              precision    recall  f1-score   support

           0       0.29      0.18      0.22       191
           1       0.00      0.00      0.00       177
           2       0.21 

/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.1

Результаты на нормализованных данных получились немного точнее.

In [94]:
# создаём объект класса логистической регрессии с SGD
log_reg_l2 = linear_model.LogisticRegression(
    penalty='l2',
    C=5,
    max_iter=1000,
    solver='sag', 
    random_state=42,
)

# обучаем модель — ищем параметры по методу SGD
log_reg_l2.fit(X_train_scaled, y_train)


LogisticRegression(C=5, max_iter=1000, random_state=42, solver='sag')

In [95]:
#Делаем предсказание для тренировочной выборки
y_train_pred = log_reg_l2.predict(X_train_scaled)
#Вывод отчет о метриках классификации
print(metrics.classification_report(y_train, y_train_pred))
#Делаем предсказание для тестовой выборки
y_test_pred = log_reg_l2.predict(X_test_scaled)
#Вывод отчет о метриках классификации
print(metrics.classification_report(y_test, y_test_pred))



              precision    recall  f1-score   support

           0       0.32      0.20      0.24       573
           1       0.17      0.00      0.00       531
           2       0.31      0.12      0.17       445
           3       0.37      0.03      0.05       343
           4       0.00      0.00      0.00        11
           5       0.17      0.02      0.04       187
           6       0.35      0.56      0.43      1276
           7       0.00      0.00      0.00        97
           8       0.36      0.61      0.45      1320
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00        50

    accuracy                           0.35      4837
   macro avg       0.19      0.14      0.13      4837
weighted avg       0.31      0.35      0.29      4837

              precision    recall  f1-score   support

           0       0.31      0.20      0.25       191
           1       0.00      0.00      0.00       177
           2       0.21 

/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.1

Метрики показывают тот же результат, что и модель с данными по умолчанию, скорее всего это связано с тем, что по умолчанию модель все же очень точно сама подбирает параметры

## Деревья решений

In [102]:
# Создаем объект класса дерево решений
dt = tree.DecisionTreeClassifier(
    criterion='gini',
    random_state=42
)

# Обучаем дерево по алгоритму CART
dt.fit(X_train_scaled, y_train)

# Выводим значения метрики
y_train_pred = dt.predict(X_train_scaled)
print(metrics.classification_report(y_train, y_train_pred))
y_test_pred = dt.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       573
           1       1.00      1.00      1.00       531
           2       1.00      1.00      1.00       445
           3       1.00      1.00      1.00       343
           4       1.00      1.00      1.00        11
           5       1.00      1.00      1.00       187
           6       1.00      1.00      1.00      1276
           7       1.00      1.00      1.00        97
           8       1.00      1.00      1.00      1320
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00        50

    accuracy                           1.00      4837
   macro avg       1.00      1.00      1.00      4837
weighted avg       1.00      1.00      1.00      4837

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       191
           1       0.84      0.85      0.84       177
           2       0.81 

Переобучение модели

In [43]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(
    n_estimators=500,
    max_depth=8,
    criterion='entropy',
    min_samples_leaf=10,
    random_state=42
)
#Обучаем модель
rf.fit(X_train_scaled, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train_scaled)
print(metrics.classification_report(y_train, y_train_pred))
y_test_pred = rf.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.63      0.41      0.50       573
           1       0.88      0.26      0.41       531
           2       0.72      0.31      0.43       445
           3       0.61      0.35      0.44       343
           4       0.00      0.00      0.00        11
           5       0.88      0.22      0.36       187
           6       0.51      0.87      0.64      1276
           7       0.00      0.00      0.00        97
           8       0.60      0.75      0.67      1320
           9       0.00      0.00      0.00         4
          10       1.00      0.08      0.15        50

    accuracy                           0.58      4837
   macro avg       0.53      0.30      0.33      4837
weighted avg       0.62      0.58      0.54      4837

              precision    recall  f1-score   support

           0       0.39      0.28      0.33       191
           1       0.82      0.13      0.22       177
           2       0.50 

/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tatiana/.local/lib/python3.1

На данном этапе ансамблевая модель показывает лучший результат. Модель предсказуемо определяет крупные квадратные предметы. Но значительно лучше по сравнению с лог. регрессией.   
Метка: 0, Значение: bed
Метка: 1, Значение: box
Метка: 2, Значение: chair
Метка: 3, Значение: cupboard
Метка: 4, Значение: frame
Метка: 5, Значение: mattress
Метка: 6, Значение: sofa
Метка: 7, Значение: stand
Метка: 8, Значение: table
Метка: 9, Значение: towel
Метка: 10, Значение: wardrobe

In [100]:

# градиентный бустинг 
gboost = ensemble.GradientBoostingClassifier(
    learning_rate=0.05, n_estimators=300, min_samples_leaf=5,
    max_depth=5, random_state=42)
gboost.fit(X_train_scaled, y_train)

y_train_pred = gboost.predict(X_train_scaled)
print(metrics.classification_report(y_train, y_train_pred, digits=3))
y_test_pred = gboost.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_test_pred, digits=3))

              precision    recall  f1-score   support

           0      1.000     1.000     1.000       573
           1      1.000     1.000     1.000       531
           2      1.000     1.000     1.000       445
           3      1.000     1.000     1.000       343
           4      1.000     1.000     1.000        11
           5      1.000     1.000     1.000       187
           6      1.000     1.000     1.000      1276
           7      1.000     1.000     1.000        97
           8      1.000     1.000     1.000      1320
           9      1.000     1.000     1.000         4
          10      1.000     1.000     1.000        50

    accuracy                          1.000      4837
   macro avg      1.000     1.000     1.000      4837
weighted avg      1.000     1.000     1.000      4837

              precision    recall  f1-score   support

           0      0.985     1.000     0.992       191
           1      0.989     1.000     0.994       177
           2      0.993 

In [101]:
# объедините уже известные вам алгоритмы с помощью стекинга 

dtree = tree.DecisionTreeClassifier(
    criterion='entropy', max_depth=7, min_samples_split=7, random_state=42)
logregr = linear_model.LogisticRegression(
    solver='sag', max_iter=1000, random_state=42)
gboost = ensemble.GradientBoostingClassifier(
    learning_rate=0.05, n_estimators=300, min_samples_leaf=5, 
    max_depth=5, random_state=42)

estimators = [
    ('dtree', dtree), ('logregr', logregr), ('gboost', gboost)]

stacked = ensemble.StackingClassifier(estimators=estimators)
stacked.fit(X_train_scaled, y_train)

y_train_pred = stacked.predict(X_train_scaled)
print(metrics.classification_report(y_train, y_train_pred, digits=3))
y_test_pred = stacked.predict(X_test_scaled)
print(metrics.classification_report(y_test, y_test_pred, digits=3))

/home/tatiana/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


              precision    recall  f1-score   support

           0      1.000     1.000     1.000       573
           1      1.000     1.000     1.000       531
           2      1.000     1.000     1.000       445
           3      1.000     1.000     1.000       343
           4      1.000     1.000     1.000        11
           5      1.000     1.000     1.000       187
           6      1.000     1.000     1.000      1276
           7      1.000     1.000     1.000        97
           8      1.000     1.000     1.000      1320
           9      1.000     1.000     1.000         4
          10      1.000     1.000     1.000        50

    accuracy                          1.000      4837
   macro avg      1.000     1.000     1.000      4837
weighted avg      1.000     1.000     1.000      4837

              precision    recall  f1-score   support

           0      0.974     1.000     0.987       191
           1      0.989     1.000     0.994       177
           2      0.993 

Испытаем модель Случайный лес на данных, которые не вошли в тренировочный и тестовый датасет для этого из filtered_df выберем строку c goal = None

In [63]:
row = filtered_df.loc[203]
row

id                                                            204
image_id                                                       15
category_id                                                    65
bbox            [56.13187026977539, 15.843657493591309, 83.130...
segmentation                                                   []
area                                                   4086.31227
file_name                     1634020258223_Sofa Set for sale.jpg
width                                                         142
height                                                        107
attributes      {'color': 'red', 'size': 'small', 'material': ...
tokens                   [1634020258223_sofa, set, for, sale.jpg]
goal                                                         None
Name: 203, dtype: object

In [64]:
row = row.drop(['segmentation', 'file_name', 'tokens', 'width', 'height', 'id', 'attributes', 'goal'])

In [65]:
row

image_id                                                      15
category_id                                                   65
bbox           [56.13187026977539, 15.843657493591309, 83.130...
area                                                  4086.31227
Name: 203, dtype: object

In [66]:
bbox_values = list(map(float, row['bbox'].strip('[]').split(',')))
row['bbox1'], row['bbox2'], row['bbox3'], row['bbox4'] = bbox_values

In [71]:
df_row = row.to_frame().T
df_row

,image_id,category_id,area,bbox1,bbox2,bbox3,bbox4
203,15,65,4086.31227,56.13187,15.843657,83.130493,49.155396


In [72]:
## предсказание на основе модели RandomForestClassifier
# делаем предсказание вероятностей
y_new_client_rf = rf.predict_proba(df_row)[:, 1]

# выводим предсказанные вероятности
print(f'Predicted probabilities: {np.round(y_new_client_rf, 2)}')

# делаем предсказание класса
y_new_client_predict_rf_clf = rf.predict(df_row)

# выводим предсказанный класс
print(f'Predicted class: {y_new_client_predict_rf_clf}')

Predicted probabilities: [0.16]
Predicted class: [8]


/home/tatiana/.local/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/home/tatiana/.local/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [73]:
# зафиксируем версию пакетов, для воспроизводимы воспроизводимости экспериментов:
!pip freeze > requirements.txt